In [39]:
import findspark
findspark.init('/home/spark/spark-2.1.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession

In [40]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [41]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection',inferSchema = True,sep = '\t')

In [42]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [43]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [44]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [45]:
from pyspark.sql.functions import length

In [46]:
data = data.withColumn('length',length(data['text']))

In [47]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [48]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [49]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer)

In [50]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [51]:
from pyspark.ml.feature import VectorAssembler

In [52]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [53]:
from pyspark.ml.classification import NaiveBayes

In [54]:
nb  = NaiveBayes()

In [55]:
from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [56]:
cleaner= data_prep_pipe.fit(data)

In [57]:
clean_data = cleaner.transform(data)
clean_data.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13458,[8,12,33,6...|(13458,[8,12,33,6...|(13459,[8,12,33,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13458,[0,26,308,...|(13458,[0,26,308,...|(13459,[0,26,308,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13458,[2,14,20,3...|(13458,[2,14,20,3...|(13459,[2,14,20,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13458,[0,73,84,1...|(13458,[0,73,8

In [58]:
clean_data = clean_data.select('label','features')

In [59]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13459,[8,12,33,6...|
|  0.0|(13459,[0,26,308,...|
|  1.0|(13459,[2,14,20,3...|
|  0.0|(13459,[0,73,84,1...|
|  0.0|(13459,[36,39,140...|
|  1.0|(13459,[11,57,62,...|
|  0.0|(13459,[11,55,108...|
|  0.0|(13459,[133,195,4...|
|  1.0|(13459,[1,50,124,...|
|  1.0|(13459,[0,1,14,29...|
|  0.0|(13459,[5,19,36,4...|
|  1.0|(13459,[9,18,40,9...|
|  1.0|(13459,[14,32,50,...|
|  0.0|(13459,[42,99,101...|
|  0.0|(13459,[567,1745,...|
|  1.0|(13459,[32,113,11...|
|  0.0|(13459,[86,224,47...|
|  0.0|(13459,[0,2,52,13...|
|  0.0|(13459,[0,77,107,...|
|  1.0|(13459,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



In [60]:
training,test = clean_data.randomSplit([0.7,0.3])

In [61]:
spam_detector = nb.fit(training)

In [62]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [63]:
test_results = spam_detector.transform(test)

In [64]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13459,[0,1,3,10,...|[-605.88926962178...|[1.0,3.3767016720...|       0.0|
|  0.0|(13459,[0,1,4,53,...|[-791.31560501209...|[1.0,2.9041485833...|       0.0|
|  0.0|(13459,[0,1,5,15,...|[-760.36822220934...|[1.0,5.3257634677...|       0.0|
|  0.0|(13459,[0,1,5,22,...|[-710.67322844529...|[1.0,9.9886360136...|       0.0|
|  0.0|(13459,[0,1,15,33...|[-219.51336545337...|[1.0,3.0331086397...|       0.0|
|  0.0|(13459,[0,1,19,21...|[-848.24804675119...|[1.0,1.2372998583...|       0.0|
|  0.0|(13459,[0,1,25,66...|[-1349.4589254148...|[1.0,9.5969816180...|       0.0|
|  0.0|(13459,[0,1,29,38...|[-1490.8550755362...|[0.99983399347298...|       0.0|
|  0.0|(13459,[0,1,156,1...|[-250.08345492570...|[0.99992597023278...|       0.0|
|  0.0|(13459,[0

In [65]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [66]:
acc_eval = MulticlassClassificationEvaluator()

In [67]:
acc = acc_eval.evaluate(test_results)

In [68]:
print("acc of model ")
print(acc)

acc of model 
0.9274226625998887
